In [19]:
import numpy as np
import pandas as pd

## IMPORT DATA

Data type should be made in a csv file in this order: 
   
ex:  
  
company_name,adv,dadv,average_salary(만원),total_sale(억원),turn_over_rate  
삼성전자, 삼성 전자의 모든 장점 리뷰 데이터, 삼성 전자의 모든 단점 리뷰 데이터, 삼성전자의 평균 연봉 (만원), 삼성전자의 연 매출액 (억원), 삼성전자의 이직율

In [20]:
df = pd.read_csv("sample_input.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df

,company_name,adv,dadv,average_salary,total_sale,turn_over_rate
0,현대카드,일단 오피스 환경이 쾌적한것은 장점대기업만의 성과급이나 복지의 혜택등이 좋다생각보다...,꼰대문화 부서에따라 존재. 굽신굽신 매일 퇴근은 정시에 포기 할일 다하면 다른업무...,8737,2300.0,0.35
1,아모레퍼시픽,자사 제품 복지몰 구매 가능(50%할인 및 100만원 상당의 포인트)다양한 업무 경...,급하게 변하는 결정들로 인해갑자기 변경되는 계획들모든 크고 작은 결정들을하나부터 열...,6999,3500.0,0.25
2,카카오,"자유로운 근무환경, 수평적인 문화, 경쟁력있는 사업영역 등 라떼는 자율 출퇴근에, ...",음 딱히..보이질 않지만.. 최근에 네이버 대비 떨어지는 경쟁력 딱히 단점이라고 할...,12717,2100.0,0.15
3,네이버,우리나라에서 가장 큰 IT기업이며 국민 서비스이자 글로벌 진출 가능성도 높습니다. ...,과거보다 조금 더딘 느낌입니다. 치킨게임으로 가고 있는 ICT산업 특성상 규모의 ...,13257,5000.0,0.32
4,네이버웹툰,"연차를 사람 눈치 안보고 쓸 수 있음, 원하는 만큼 공부하고 성장할 수 있음, 구내...",경쟁률이 너무 심하다보니 인턴의 경우 전환률이 너무 낮음. 애초에 팀에 티오도 별로...,5305,1610.2,0.28


## Topic Modeling

create topic modeling result

In [21]:
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
from gensim.models import KeyedVectors

import pickle

model_adv = KeyedVectors.load("tp_model/tp_adv/tp_adv_model")
model_dadv = KeyedVectors.load("tp_model/tp_dadv/tp_dadv_model")

with open('input/corpus_adv.pkl', 'rb') as lf:
    corpus_adv = pickle.load(lf)
with open('input/corpus_dadv.pkl', 'rb') as lf:
    corpus_dadv = pickle.load(lf)

In [22]:
num_topics_adv = 6
num_topics_dadv = 7

output_df_adv = pd.DataFrame({'company_name':df.company_name})
for col in range(num_topics_adv):
    tmp = str(col)
    output_df_adv[tmp] = float(0)
output_df_adv.index = [i for i in range(len(output_df_adv))]

output_df_dadv = pd.DataFrame({'company_name':df.company_name})
for col in range(num_topics_dadv):
    tmp = str(col)
    output_df_dadv[tmp] = float(0)
output_df_dadv.index = [i for i in range(len(output_df_dadv))]

for i, dt in enumerate(model_adv.get_document_topics(corpus_adv)):
    for val in dt:

        idx = str(val[0])
        pt = val[1]

        output_df_adv.at[i,idx] = pt

for i, dt in enumerate(model_dadv.get_document_topics(corpus_dadv)):
    for val in dt:

        idx = str(val[0])
        pt = val[1]

        output_df_dadv.at[i,idx] = pt

In [23]:
# output_df

df = df.drop(['adv','dadv'], axis = 1)

# adv
output_df_adv = output_df_adv.dropna()

col_adv = list(output_df_adv.columns)

for i in range(1, len(col_adv)):
    col_adv[i] = "adv_topic_"+col_adv[i]
output_df_adv.columns = col_adv

# dadv
output_df_dadv = output_df_dadv.dropna()

col_dadv = list(output_df_dadv.columns)

for i in range(1, len(col_dadv)):
    col_dadv[i] = "dadv_topic_"+col_dadv[i]
output_df_dadv.columns = col_dadv

output_df = pd.merge(output_df_adv, output_df_dadv ,how='inner')
output_df = pd.merge(output_df, df ,how='inner')
output_df

,company_name,adv_topic_0,adv_topic_1,adv_topic_2,adv_topic_3,adv_topic_4,adv_topic_5,dadv_topic_0,dadv_topic_1,dadv_topic_2,dadv_topic_3,dadv_topic_4,dadv_topic_5,dadv_topic_6,average_salary,total_sale,turn_over_rate
0,현대카드,0.978254,0.000000,0.010190,0.000000,0.000000,0.000000,0.000000,0.073691,0.000000,0.013542,0.000000,0.912490,0.000000,8737,2300.0,0.35
1,아모레퍼시픽,0.711998,0.000000,0.000000,0.242970,0.044643,0.000000,0.000000,0.224451,0.000000,0.000000,0.000000,0.775140,0.000000,6999,3500.0,0.25
2,카카오,0.544925,0.000000,0.190050,0.000000,0.264180,0.000000,0.084182,0.229913,0.022778,0.021380,0.000000,0.631737,0.000000,12717,2100.0,0.15
3,네이버,0.156598,0.123652,0.000000,0.294278,0.000000,0.420747,0.000000,0.514205,0.356452,0.000000,0.097374,0.000000,0.029307,13257,5000.0,0.32
4,네이버웹툰,0.822690,0.000000,0.176463,0.000000,0.000000,0.000000,0.000000,0.258842,0.000000,0.000000,0.071889,0.656891,0.000000,5305,1610.2,0.28


In [24]:
# log transformation of output_df

for feature in list(output_df.columns)[1:-1]:
    if feature != 'dadv_topic_4':
        nonzero_indices = output_df[feature] != 0  # 0이 아닌 값의 인덱스를 찾음
        output_df[feature] = np.log1p(output_df[feature])
output_df = output_df.rename(columns={"average_salary":"average_salary(만원)", "total_sale":"total_sale(억원)"})
output_df

,company_name,adv_topic_0,adv_topic_1,adv_topic_2,adv_topic_3,adv_topic_4,adv_topic_5,dadv_topic_0,dadv_topic_1,dadv_topic_2,dadv_topic_3,dadv_topic_4,dadv_topic_5,dadv_topic_6,average_salary(만원),total_sale(억원),turn_over_rate
0,현대카드,0.682214,0.000000,0.010139,0.000000,0.000000,0.000000,0.000000,0.071102,0.000000,0.013451,0.000000,0.648406,0.000000,9.075437,7.741099,0.35
1,아모레퍼시픽,0.537661,0.000000,0.000000,0.217504,0.043675,0.000000,0.000000,0.202493,0.000000,0.000000,0.000000,0.573879,0.000000,8.853665,8.160804,0.25
2,카카오,0.434975,0.000000,0.173995,0.000000,0.234424,0.000000,0.080826,0.206944,0.022522,0.021154,0.000000,0.489645,0.000000,9.450774,7.650169,0.15
3,네이버,0.145483,0.116584,0.000000,0.257953,0.000000,0.351183,0.000000,0.414890,0.304873,0.000000,0.097374,0.000000,0.028886,9.492356,8.517393,0.32
4,네이버웹툰,0.600313,0.000000,0.162512,0.000000,0.000000,0.000000,0.000000,0.230192,0.000000,0.000000,0.071889,0.504943,0.000000,8.576594,7.384735,0.28


run it into machine learning

In [25]:
import joblib
lgbm_model = joblib.load('ml_model/tp_lgbm_reg_model.pkl')

y_target = output_df['turn_over_rate']
x_data = output_df.drop(['company_name','turn_over_rate'], axis = 1, inplace = False)

print("Predicted turnover rate of the company using lgbm model is :",lgbm_model.predict(x_data))
print("Real turnover of the company is : ", y_target.to_list())

Predicted turnover rate of the company using lgbm model is : [0.23826028 0.23787881 0.18965985 0.26083969 0.31589772]
Real turnover of the company is :  [0.35, 0.25, 0.15, 0.32, 0.28]


In [26]:
xgb_model = joblib.load('ml_model/tp_xgb_reg_model.pkl')

y_target = output_df['turn_over_rate']
x_data = output_df.drop(['company_name','turn_over_rate'], axis = 1, inplace = False)

print("Predicted turnover rate of the company using xgb model is :",xgb_model.predict(x_data))
print("Real turnover of the company is : ", y_target.to_list())

Predicted turnover rate of the company using xgb model is : [0.2879109  0.28654602 0.2867715  0.35204157 0.34026417]
Real turnover of the company is :  [0.35, 0.25, 0.15, 0.32, 0.28]


In [27]:
tp_df = pd.read_csv('tp_df.csv')
tp_df = tp_df.drop(["Unnamed: 0","company_name","turn_over_rate","average_salary"], axis=1)

tmp = output_df.drop(["company_name","turn_over_rate","average_salary(만원)","total_sale(억원)"], axis=1)

summary = tp_df.describe()
summary = summary.drop(["count","std"],axis = 0)

In [28]:
categories = ["mean", "min","25%","50%","75%","max"]

for i in range(len(summary.columns)):
    save = []
    # print(tmp.iloc[0,i])
    
    for j in range (len(summary.iloc[:,i])):
        if tmp.iloc[0,i] > summary.iloc[j,i]:
            save.append(categories[j])
    if len(save) >0:
        print("For topic: ",tmp.columns[i])
        if save[0] == "mean":
            print("The topic modeling value is greater than mean and ", save[-1] ,"of data")
        else:
            print("The topic modeling value is greater than ", save[-1],"of data")

For topic:  adv_topic_0
The topic modeling value is greater than mean and  75% of data
For topic:  adv_topic_2
The topic modeling value is greater than  50% of data
For topic:  dadv_topic_1
The topic modeling value is greater than  50% of data
For topic:  dadv_topic_3
The topic modeling value is greater than  50% of data
For topic:  dadv_topic_5
The topic modeling value is greater than mean and  75% of data
